In [2]:
import requests
import math
import csv
import time

In [9]:
#Write the results line by line to a csv file
def toFile(out_file, results):
    with open(out_file + '.csv', "w", encoding="utf-8") as o:
        write = csv.writer(o)
        write.writerows(results)
    

In [4]:
# Extract the required information from the json object
def extract(response):
    results = []
    for result in response.json()['result']:
        results.append([result['url'], result['headline'], result['firstPublishDate'], result['body'][:350]])
    return results

In [5]:
def get_news(query, out_file):
    #It's the API that CNN uses to get their data. 
    #We use this just to avoid rendering Javascript.
    url = 'https://search.api.cnn.io/content'
    #The first query to get the first 50 results
    params = { 'q' : query ,'size' : 50, 'page' : 1}
    response = requests.get(url = url, params = params)
    #We get the total number of results, so now we know how many more pages we have to request.
    #We round up the result after the division
    pages = response.json()['meta']['of']/50
    roundedUp = int(math.ceil(pages))
    results = extract(response)
    for page in range(2, roundedUp+1):
        fromPage = (page - 1) * 50
        params = { 'q' : query ,'size' : 50, 'from' :  fromPage, 'page' : page}
        #Sleep for 0.5 second just for safety :) 
        time.sleep(0.5)
        response = requests.get(url = url, params = params)
        results += extract(response)
    toFile(out_file, results)


In [10]:
get_news('holiday greece', 'holiday_greece')